In [1]:
from app.template.variable.expr import LazyExpr
from typing import List, Dict, Any, Optional
from datetime import datetime
import ast

In [2]:
im = ["import numpy as np", "from datetime import datetime"]

# s = "lambda x: all(isinstance(i, datetime) for i in x) if isinstance(x, list) else False"
s = "(aaaaa := a + 1)"
# s = """
# 1+1
# """
f = LazyExpr(s, im)

# print(ast.dump(f._ast, indent=2))
print(f.unbound)
print(f.target)

f.eval(env={"a": 1024})

{'a'}
aaaaa


1025

In [16]:
from app.template.data_source.model import DictSource

s = DictSource(name="test",
               data={
                   "a": 1,
                   "b": 2,
                   "c": 3
               },
               json_schema={
                   "type": "object",
                   "properties": {
                       "a": {
                           "type": "integer"
                       },
                       "b": {
                           "type": "integer"
                       },
                       "c": {
                           "type": "integer"
                       }
                   }
               })
s.load()

Ok({'a': 1, 'b': 2, 'c': 3})

In [4]:
from app.template.variable.model import IVariable, LiteralVariable

LiteralVariable(name="s", expr=LazyExpr("1+1"))

LiteralVariable(name='s', expr=LazyExpr(raw='1+1', imports=[]), comment=None, formatter=None, t=None)

In [5]:
var_a = LiteralVariable.model_validate({
    "name": "a",
    "expr": "datetime.now()",
    "t": "int",
    "imports": ["from datetime import datetime"],
})

var_a.load()

Err(ValueError('failed to verify value: a', TypeCheckError('is not an instance of int')))

In [6]:
# var_s = LiteralVariable(name="s", expr=LazyExpr("datetime.now()"), t=datetime, imports=im)
var_s = LiteralVariable(name="s",
                        expr="datetime.now()",
                        formatter="lambda x: x.strftime('%Y-%m-%d %H:%M:%S')",
                        imports=im,
                        t=datetime)
identity = lambda x: x
display(var_s.model_dump())
var_s_val = var_s.load().unwrap()
format_fn = var_s.eval_formatter()
assert format_fn is not None
display(format_fn(var_s_val))

{'name': 's',
 'expr': {'raw': 'datetime.now()',
  'imports': ['import numpy as np', 'from datetime import datetime']},
 'comment': None,
 'formatter': {'raw': "lambda x: x.strftime('%Y-%m-%d %H:%M:%S')",
  'imports': ['import numpy as np', 'from datetime import datetime']},
 't': datetime.datetime}

'2024-03-28 00:08:56'

In [7]:
lst = [
    LiteralVariable(name="s", expr=LazyExpr("1+1")),
    LiteralVariable(name="a", expr=LazyExpr("1+1")),
    LiteralVariable(name="b", expr=LazyExpr("a+1")),
    LiteralVariable(name="c", expr=LazyExpr("b+1")),
    LiteralVariable(name="d", expr=LazyExpr("a+b+2")),
    LiteralVariable(name="e", expr=LazyExpr("c+d+3")),
]
for v in lst:
    print(v.unbound)

set()
set()
{'a'}
{'b'}
{'b', 'a'}
{'d', 'c'}


In [8]:
from app.template.dependency.resolver import resolve_and_evaluate, to_env_dict

vars = resolve_and_evaluate(lst)
if vars.is_ok():
    display(to_env_dict(vars.unwrap()))

{'a': 2, 'b': 3, 'c': 4, 'd': 7, 'e': 14, 's': 2}

In [9]:
text = "本期中出院病例${{'a':0}}例。手术人数${num_operation}人，四级手术${num_operation_4}人，微创手术${num_operation_micro}人。CMI值${CMI}，其中 CMI 小于1的病人数有${query(CMI<1)}人，占比${query(CMI<1)/num_patient}；1-2的病人数${query(CMI > 1 and CMI < 2)}人，占比${query(CMI > 1 and CMI < 2) / num_patient};2-5的病人数${(num_CIM_2_5 := query(CMI > 2 and CMI < 5))}人，占比${num_CIM_2_5 / num_patient};大于5的病人数${(num_CIM_gt_5 := query(CMI > 5))}人,占比${num_CIM_gt_5 / num_patient}。总体来说大于2的病人数${(num_CIM_gt_2 := query(CMI > 2))}人，占比${num_CIM_gt_2 / num_patient}%，有提升空间。"

In [11]:
from app.template.dependency.extractor import parse

parsed = parse(text)

# print(parsed)

display(parsed.unwrap().text)
display(parsed.unwrap().table)

'本期中出院病例#938344a3#例。手术人数#e2264901#人，四级手术#d5191cf5#人，微创手术#272aea71#人。CMI值#50c182b7#，其中 CMI 小于1的病人数有#44224433#人，占比#9b57cb36#；1-2的病人数#0bc37be4#人，占比#0c46628a#;2-5的病人数#80ec94a6#人，占比#f6d11d05#;大于5的病人数#0919f947#人,占比#682db670#。总体来说大于2的病人数#023ed6d7#人，占比#c33640e8#%，有提升空间。'

[('938344a3', "{'a':0}"),
 ('e2264901', 'num_operation'),
 ('d5191cf5', 'num_operation_4'),
 ('272aea71', 'num_operation_micro'),
 ('50c182b7', 'CMI'),
 ('44224433', 'query(CMI<1)'),
 ('9b57cb36', 'query(CMI<1)/num_patient'),
 ('0bc37be4', 'query(CMI > 1 and CMI < 2)'),
 ('0c46628a', 'query(CMI > 1 and CMI < 2) / num_patient'),
 ('80ec94a6', '(num_CIM_2_5 := query(CMI > 2 and CMI < 5))'),
 ('f6d11d05', 'num_CIM_2_5 / num_patient'),
 ('0919f947', '(num_CIM_gt_5 := query(CMI > 5))'),
 ('682db670', 'num_CIM_gt_5 / num_patient'),
 ('023ed6d7', '(num_CIM_gt_2 := query(CMI > 2))'),
 ('c33640e8', 'num_CIM_gt_2 / num_patient')]